In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/soundandaccel-labelledcnc/CNC_Labelled_Data_Cleaned.parquet


In [2]:
#Loading of the dataset for the study
import pyarrow as pa 

DfLabelledTimeSeriesCleaned=pd.read_parquet('/kaggle/input/soundandaccel-labelledcnc/CNC_Labelled_Data_Cleaned.parquet', engine='pyarrow')

In [3]:
import dask.dataframe as dd
import os 
import csv

outPath='/kaggle/working/myOut'
os.makedirs(outPath, exist_ok=True)

In [4]:
DfLabelledTimeSeriesCleaned=DfLabelledTimeSeriesCleaned.drop(columns=['frame', 'frame_time_ms', 'frame_duration_ms', 'Op_Nb','Label_Program_Line', 'Label_Op_Name',])

In [5]:
DfLabelledTimeSeriesCleaned.head()

,Data_points,Sound,Accel X,Accel Y,Accel Z,Label_Tool_Name
0,8335,"[147, 151, 174, 195, 218, 248, 278, 314, 381, ...","[160, 216, 254, 177, -19, -154, -191, -149, 16...","[104, 147, 124, 116, 133, 158, 182, 138, 118, ...","[-101, -156, 108, 263, 40, -249, -215, 69, 41,...",T07 M06 (MECHE MD D4.2)
1,8336,"[404, 407, 402, 408, 436, 466, 488, 490, 471, ...","[-586, -546, -477, -368, -154, 104, 170, 54, -...","[24, -4, -126, -244, -203, -70, 13, -11, -104,...","[585, 118, -278, -296, -61, 327, 488, 271, 26,...",T07 M06 (MECHE MD D4.2)
2,8335,"[509, 564, 583, 564, 534, 526, 537, 532, 483, ...","[-591, -658, -488, -166, 293, 653, 709, 654, 6...","[-44, -3, 26, 1, -3, 2, -11, -2, 41, 101, 57, ...","[406, -296, -236, 218, 67, -293, -66, 492, 474...",T07 M06 (MECHE MD D4.2)
3,8335,"[356, 313, 336, 385, 432, 460, 468, 472, 488, ...","[-175, -413, -427, -273, -132, -59, 142, 477, ...","[169, 188, 184, 143, 118, 58, 52, 158, 202, 17...","[-78, -51, 5, -264, -568, -221, 185, 84, 175, ...",T07 M06 (MECHE MD D4.2)
4,8336,"[349, 330, 304, 296, 301, 317, 331, 335, 323, ...","[442, 120, -181, -478, -737, -650, -187, 241, ...","[-65, -68, -17, -20, -71, -100, -68, -68, -116...","[776, 730, -160, -527, 27, 237, -268, -347, -5...",T07 M06 (MECHE MD D4.2)


In [6]:


file_count = 1
mySnd = []
myAccX=[]
myAccY=[]
myAccZ=[]
current_label = None
segmentsPerWindow=30
j=0
l=0


for i, row in DfLabelledTimeSeriesCleaned.iterrows():


    if not mySnd:  # Première ligne
        current_label = row['Label_Tool_Name']
        l+=1 #label number
        myDir=os.path.join(outPath, f"Label_{l}_{current_label}")
        os.makedirs(myDir, exist_ok=True)

    if row['Label_Tool_Name'] != current_label or j >= segmentsPerWindow:
        j=0
        outFileSnd=os.path.join(myDir, f"{file_count:04d}_sound_label_{current_label}.csv")
        outFileAccX=os.path.join(myDir, f"{file_count:04d}_accX_label_{current_label}.csv")
        outFileAccY=os.path.join(myDir, f"{file_count:04d}_accY_label_{current_label}.csv")
        outFileAccZ=os.path.join(myDir, f"{file_count:04d}_accZ_label_{current_label}.csv")
        with open(outFileSnd, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(mySnd)
        with open(outFileAccX, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(myAccX)
        with open(outFileAccY, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(myAccY)
        with open(outFileAccZ, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(myAccZ)
        
        print(f"Fichier sauvegardé : {file_count}_{myDir}")
        if row['Label_Tool_Name'] != current_label:
            l+=1
            myDir=os.path.join(outPath, f"Label_{l}_{row['Label_Tool_Name']}")
            os.makedirs(myDir, exist_ok=True)

        mySnd=[]
        myAccX=[]
        myAccY=[]
        myAccZ=[]
        current_label = row['Label_Tool_Name']
        file_count += 1
    j+=1
    mySnd.extend(list(row['Sound']))
    myAccX.extend(list(row['Accel X']))
    myAccY.extend(list(row['Accel Y']))
    myAccZ.extend(list(row['Accel Z']))

if mySnd:
    myDir=os.path.join(outPath, f"Label_{l}_{current_label}")
    os.makedirs(myDir, exist_ok=True)
    outFileSnd=os.path.join(myDir, f"{file_count:04d}_sound_label_{current_label}.csv")
    outFileAccX=os.path.join(myDir, f"{file_count:04d}_accX_label_{current_label}.csv")
    outFileAccY=os.path.join(myDir, f"{file_count:04d}_accY_label_{current_label}.csv")
    outFileAccZ=os.path.join(myDir, f"{file_count:04d}_accZ_label_{current_label}.csv")
    with open(outFileSnd, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(mySnd)
    with open(outFileAccX, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(myAccX)
    with open(outFileAccY, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(myAccY)
    with open(outFileAccZ, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(myAccZ)
    print(f"Fichier sauvegardé : {myDir}")




Fichier sauvegardé : 1_/kaggle/working/myOut/Label_1_T07 M06 (MECHE MD D4.2)
Fichier sauvegardé : 2_/kaggle/working/myOut/Label_1_T07 M06 (MECHE MD D4.2)
Fichier sauvegardé : 3_/kaggle/working/myOut/Label_1_T07 M06 (MECHE MD D4.2)
Fichier sauvegardé : 4_/kaggle/working/myOut/Label_1_T07 M06 (MECHE MD D4.2)
Fichier sauvegardé : 5_/kaggle/working/myOut/Label_1_T07 M06 (MECHE MD D4.2)
Fichier sauvegardé : 6_/kaggle/working/myOut/Label_2_T02 M06 (VHM 309-12)
Fichier sauvegardé : 7_/kaggle/working/myOut/Label_2_T02 M06 (VHM 309-12)
Fichier sauvegardé : 8_/kaggle/working/myOut/Label_2_T02 M06 (VHM 309-12)
Fichier sauvegardé : 9_/kaggle/working/myOut/Label_2_T02 M06 (VHM 309-12)
Fichier sauvegardé : 10_/kaggle/working/myOut/Label_2_T02 M06 (VHM 309-12)
Fichier sauvegardé : 11_/kaggle/working/myOut/Label_2_T02 M06 (VHM 309-12)
Fichier sauvegardé : 12_/kaggle/working/myOut/Label_2_T02 M06 (VHM 309-12)
Fichier sauvegardé : 13_/kaggle/working/myOut/Label_2_T02 M06 (VHM 309-12)
Fichier sauvegardé 

In [7]:
# Gather the file duration based on the sampling frequency
def get_file_duration(file_path, fs):
    try:
        ext = os.path.splitext(file_path)[1]

        # For .csv files
        if ext in [".csv"]:
           with open(file_path, 'r', newline='', encoding="utf-8") as csvfile:
            raw_data = csvfile.read().replace("\r\n", "")  # Supprimer les retours à la ligne
            myReader= list(csv.reader([raw_data], delimiter=','))
            return  float(len(myReader[0])/fs) #Duration in seconds

        # Other types of files
        else:
            return None  # Pas de durée disponible

    except Exception as e:
        print(f"Error with : {file_path}: {e}")
        return None

In [8]:
#Prepare a dataframe with the list of files and their corresponding labels and the duration of each observation

import os
import csv
from pathlib import Path
import pandas as pd

# 📌 Dossier à scanner (modifie ce chemin selon ton projet)
base_dir = "/kaggle/working/myOut"

# 📌 Scanner les fichiers du dossier et sous-dossiers
myDirs=[]
xs=[]
ys=[]
zs=[]
snds=[]
nbs=[]
labels=[]
durs=[]
j=1
fs=50000
lastnb=None
for root, dirs, files in os.walk(base_dir):
    
    for file in sorted(files):
        measName=file.split("_")[1]
        nb=int(file.split("_")[0])
        if nb != lastnb or lastnb == None:
            file_path = Path(root) / file
            myDir='/'+str(file_path).split('myOut/')[-1].split('/')[0]+'/'
            label=myDir.split('Label_')[-1]
            durs.append(get_file_duration(file_path, fs))
            labels.append(label)
            myDirs.append(myDir)
            nbs.append(nb)
            print(j, nb, file)
            j+=1
        if measName=='sound':
            snds.append(file)
        if measName=='accX':
            xs.append(file)
        if measName=='accY':
            ys.append(file)
        if measName=='accZ':
            zs.append(file)

        lastnb=nb

del lastnb, nb, j, root, myDir, measName, file, file_path, dirs, files, label

# 📌 Créer un DataFrame avec les informations
df = pd.DataFrame({'nb':nbs, 'dir':myDirs, 'label':labels, 'sound':snds, 'accelX':xs, 'accelY':ys, 'accelZ':zs, 'duration': durs})
del labels, nbs, myDirs, snds, xs, ys, zs, durs, fs

1 713 0713_accX_label_T24 M06 (FRSURF50X5).csv
2 714 0714_accX_label_T24 M06 (FRSURF50X5).csv
3 715 0715_accX_label_T24 M06 (FRSURF50X5).csv
4 716 0716_accX_label_T24 M06 (FRSURF50X5).csv
5 717 0717_accX_label_T24 M06 (FRSURF50X5).csv
6 718 0718_accX_label_T24 M06 (FRSURF50X5).csv
7 719 0719_accX_label_T24 M06 (FRSURF50X5).csv
8 720 0720_accX_label_T24 M06 (FRSURF50X5).csv
9 721 0721_accX_label_T24 M06 (FRSURF50X5).csv
10 722 0722_accX_label_T24 M06 (FRSURF50X5).csv
11 723 0723_accX_label_T24 M06 (FRSURF50X5).csv
12 724 0724_accX_label_T24 M06 (FRSURF50X5).csv
13 725 0725_accX_label_T24 M06 (FRSURF50X5).csv
14 726 0726_accX_label_T24 M06 (FRSURF50X5).csv
15 727 0727_accX_label_T24 M06 (FRSURF50X5).csv
16 728 0728_accX_label_T24 M06 (FRSURF50X5).csv
17 729 0729_accX_label_T24 M06 (FRSURF50X5).csv
18 730 0730_accX_label_T24 M06 (FRSURF50X5).csv
19 731 0731_accX_label_T24 M06 (FRSURF50X5).csv
20 732 0732_accX_label_T24 M06 (FRSURF50X5).csv
21 733 0733_accX_label_T24 M06 (FRSURF50X5).csv
2

In [9]:
df.head()

#print(f"nb: {len(nbs)}, 'dir':{len(myDirs)}, 'label':{len(labels)}, 'sound':{len(snds)}, 'accelX':{len(xs)}, 'accelY':{len(ys)}, 'accelZ':{len(zs)}, 'duration': {len(durs)}")




,nb,dir,label,sound,accelX,accelY,accelZ,duration
0,713,/Label_9_T24 M06 (FRSURF50X5)/,9_T24 M06 (FRSURF50X5)/,0713_sound_label_T24 M06 (FRSURF50X5).csv,0713_accX_label_T24 M06 (FRSURF50X5).csv,0713_accY_label_T24 M06 (FRSURF50X5).csv,0713_accZ_label_T24 M06 (FRSURF50X5).csv,5.00124
1,714,/Label_9_T24 M06 (FRSURF50X5)/,9_T24 M06 (FRSURF50X5)/,0714_sound_label_T24 M06 (FRSURF50X5).csv,0714_accX_label_T24 M06 (FRSURF50X5).csv,0714_accY_label_T24 M06 (FRSURF50X5).csv,0714_accZ_label_T24 M06 (FRSURF50X5).csv,5.00122
2,715,/Label_9_T24 M06 (FRSURF50X5)/,9_T24 M06 (FRSURF50X5)/,0715_sound_label_T24 M06 (FRSURF50X5).csv,0715_accX_label_T24 M06 (FRSURF50X5).csv,0715_accY_label_T24 M06 (FRSURF50X5).csv,0715_accZ_label_T24 M06 (FRSURF50X5).csv,5.00124
3,716,/Label_9_T24 M06 (FRSURF50X5)/,9_T24 M06 (FRSURF50X5)/,0716_sound_label_T24 M06 (FRSURF50X5).csv,0716_accX_label_T24 M06 (FRSURF50X5).csv,0716_accY_label_T24 M06 (FRSURF50X5).csv,0716_accZ_label_T24 M06 (FRSURF50X5).csv,5.00124
4,717,/Label_9_T24 M06 (FRSURF50X5)/,9_T24 M06 (FRSURF50X5)/,0717_sound_label_T24 M06 (FRSURF50X5).csv,0717_accX_label_T24 M06 (FRSURF50X5).csv,0717_accY_label_T24 M06 (FRSURF50X5).csv,0717_accZ_label_T24 M06 (FRSURF50X5).csv,5.00122


In [10]:
import shutil
shutil.make_archive('/kaggle/working/myOutFolder', 'zip', '/kaggle/working/myOut')

'/kaggle/working/myOutFolder.zip'

In [11]:
!rm -r /kaggle/working/myOut

In [12]:
df.to_csv('./metadata.csv', index=False) 